<a href="https://colab.research.google.com/github/sgundala/RAG-from-Scratch/blob/main/RAG_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
if "COLAB_GPU" in os.environ:
  print("running on Colab GPU")
  !pip install PyMuPDF
  !pip install torch
  !pip install tqdm
  !pip install sentence-transformers
  !pip install accelerate
  !pip install bitsandbytes
  !pip install flash-attn --no-build-isolation

running on Colab GPU
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 141.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [24]:
!nvidia-smi


Wed Nov 12 15:05:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [25]:
!nvidia-smi || echo "No GPU found"

Wed Nov 12 15:05:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [26]:
import fitz

In [27]:
from tqdm.auto import tqdm

In [28]:
PDF_PATH = "/content/Human-Nutrition.pdf"  # update if your filename differs


In [29]:
import fitz  # PyMuPDF
from tqdm.auto import tqdm
import re

def text_formatter(text: str) -> str:
    if not text:
        return ""
    # keep it simple: flatten whitespace
    text = text.replace("\r", "\n").replace("\t", " ")
    text = " ".join(text.split()).strip()
    return text

def open_and_read_pdf(pdf_path: str, skip_first_pages: int = 41):
    """
    Returns a list of dicts, one per kept page:
      {
        "Page_number": <int>,                 # starts at 1 after skipping
        "Page_char_count": <int>,
        "page_word_count": <int>,
        "page_sentence_count_raw": <int>,     # split on . ! ?
        "page_token_count": <int>,            # ~heuristic: 1 token ≈ 4 chars
        "text": <str>
      }
    """
    doc = fitz.open(pdf_path)
    pages_and_texts = []

    # enumerate from 1 to match human page numbers
    for i, page in tqdm(enumerate(doc, start=1), total=len(doc), desc="Reading pages"):
        # skip front matter
        if i <= skip_first_pages:
            continue

        text = page.get_text("text")
        text = text_formatter(text)

        # simple counts (fast, no extra deps)
        char_count = len(text)
        word_count = len(text.split())
        sent_count = len([s for s in re.split(r"[.!?]+", text) if s.strip()])
        token_count = char_count // 4  # heuristic; swap for tiktoken if you like

        pages_and_texts.append({
            "Page_number": i - skip_first_pages,     # adjusted: first kept page -> 1
            "Page_char_count": char_count,
            "page_word_count": word_count,
            "page_sentence_count_raw": sent_count,
            "page_token_count": token_count,
            "text": text
        })

    doc.close()
    return pages_and_texts

# ---- run it ----
pdf_path = "/content/Human-Nutrition.pdf"
pages_and_texts = open_and_read_pdf(pdf_path, skip_first_pages=41)

# preview first 2 kept pages
pages_and_texts[:2]


Reading pages:   0%|          | 0/1208 [00:00<?, ?it/s]

[{'Page_number': 1,
  'Page_char_count': 745,
  'page_word_count': 117,
  'page_sentence_count_raw': 4,
  'page_token_count': 186,
  'text': 'Special Thanks to Bill Chismar – University of Hawai’i at Mānoa, Dean of Outreach College The Children’s Healthy Living Summer Institute – University of Hawai’i at Mānoa Open Educational Resources This text is provided to you as an Open Educational Resource (OER) which you access online. It is designed to give you a comprehensive introduction to human nutrition at no or very nominal cost. It contains both written and graphic text material, intra-text links to other internal material which may aid in understanding topics and concepts, intra-text links to the appendices and glossary for tables and definitions of words, and extra-text links to videos and web material that clarifies and augments topics and concepts. xlii | Acknowledgements'},
 {'Page_number': 2,
  'Page_char_count': 88,
  'page_word_count': 16,
  'page_sentence_count_raw': 3,
  'page

In [30]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df.head()

,Page_number,Page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,1,745,117,4,186,Special Thanks to Bill Chismar – University of...
1,2,88,16,3,22,PART I CHAPTER 1. BASIC CONCEPTS IN NUTRITION ...
2,3,0,0,0,0,
3,4,246,43,2,61,Image by Jim Hollyer / CC BY 4.0 Introduction ...
4,5,1155,176,10,288,Learning Objectives By the end of this chapter...


In [31]:
df.describe().round(2)

,Page_number,Page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1167.00,1167.00,1167.00,1167.00,1167.00
mean,584.00,1136.94,175.04,14.84,283.86
std,337.03,551.68,86.48,9.51,137.90
min,1.00,0.00,0.00,0.00,0.00
25%,292.50,745.00,114.00,9.00,186.00
50%,584.00,1214.00,189.00,13.00,303.00
75%,875.50,1586.00,241.00,20.00,396.00
max,1167.00,2271.00,393.00,82.00,567.00


In [15]:
import os
if "COLAB_GPU" in os.environ:
  print("running on Colab GPU")
else:
  print("running on CPU")
  !pip install PyMuPDF #for reading PDFs with python
  !pip install tqdm #for progress bars
  !pip install sentence-transformers #for embedding models
  !pip install accelerate #for quantization model loading
  !pip install bitsandbytes #for quantizing models (less storahge space)
  !pip install flash-attn --no-build-isolation #for faster attention mechanism = faster LLm inference

running on Colab GPU


In [32]:
from tqdm.auto import tqdm

# ---------- 1️⃣ Split text into chunks ---------- #
def chunk_text(text: str, chunk_size: int = 500) -> list:
    """
    Splits text into chunks of approximately `chunk_size` characters.
    Returns a list of text chunks.
    """
    chunks = []
    current_chunk = ''
    words = text.split()

    for word in words:
        # Check if adding the word exceeds chunk size
        if len(current_chunk) + len(word) + 1 <= chunk_size:
            current_chunk += word + ' '
        else:
            # Store current chunk and start a new one
            chunks.append(current_chunk.strip())
            current_chunk = word + ' '

    # Add the last chunk if not empty
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks


# ---------- 2️⃣ Split all PDF pages into chunks ---------- #
def chunk_pdf_pages(pages_and_texts: list, chunk_size: int = 500) -> list[dict]:
    """
    Takes PDF pages (list of dicts) and splits each page text into chunks.
    Returns a list of dicts containing page_number, chunk_index, and chunk_text.
    """
    all_chunks = []
    for page in tqdm(pages_and_texts, desc="Chunking pages"):
        page_number = page["Page_number"]
        page_text = page["text"]

        # Split current page into chunks
        chunks = chunk_text(page_text, chunk_size=chunk_size)

        # Record each chunk with metadata
        for i, chunk in enumerate(chunks):
            all_chunks.append({
                "page_number": page_number,
                "chunk_index": i + 1,
                "chunk_char_count": len(chunk),
                "chunk_word_count": len(chunk.split()),
                "chunk_token_count": len(chunk) // 4,  # rough token estimate
                "chunk_text": chunk
            })

    return all_chunks


# ---------- 3️⃣ Example usage ---------- #
chunked_pages = chunk_pdf_pages(pages_and_texts, chunk_size=500)

print(f"Total chunks: {len(chunked_pages)}")
print(f"First chunk (Page {chunked_pages[0]['page_number']}):\n")
print(chunked_pages[0]['chunk_text'][:400] + "...")


Chunking pages:   0%|          | 0/1167 [00:00<?, ?it/s]

Total chunks: 3253
First chunk (Page 1):

Special Thanks to Bill Chismar – University of Hawai’i at Mānoa, Dean of Outreach College The Children’s Healthy Living Summer Institute – University of Hawai’i at Mānoa Open Educational Resources This text is provided to you as an Open Educational Resource (OER) which you access online. It is designed to give you a comprehensive introduction to human nutrition at no or very nominal cost. It conta...


In [33]:
import pandas as pd
df = pd.DataFrame(chunked_pages)
df.head()

,page_number,chunk_index,chunk_char_count,chunk_word_count,chunk_token_count,chunk_text
0,1,1,497,80,124,Special Thanks to Bill Chismar – University of...
1,1,2,247,37,61,"aid in understanding topics and concepts, intr..."
2,2,1,88,16,22,PART I CHAPTER 1. BASIC CONCEPTS IN NUTRITION ...
3,4,1,246,43,61,Image by Jim Hollyer / CC BY 4.0 Introduction ...
4,5,1,498,79,124,Learning Objectives By the end of this chapter...


In [34]:
def _pack_units(units: list[str], max_tokens: int) -> list[list[str]]:
    packs, cur, cur_toks = [], [], 0
    for u in units:
        t = token_len(u)
        if cur and cur_toks + t > max_tokens:
            packs.append(cur)
            cur, cur_toks = [u], t
        else:
            cur.append(u); cur_toks += t
    if cur:
        packs.append(cur)
    return packs

def chunk_text_recursive(text: str, max_tokens: int = 500, overlap_tokens: int = 50) -> list:
    out = []
    paras = split_paragraphs(text)
    for para in paras:
        if token_len(para) <= max_tokens:
            out.append(para)
            continue
        # 1) try sentences
        sents = split_sentences(para)
        if all(token_len(s) <= max_tokens for s in sents):
            for pack in _pack_units(sents, max_tokens):
                out.append(" ".join(pack))
            continue
        # 2) some sentences are huge: fall back to words inside those
        for s in sents:
            if token_len(s) <= max_tokens:
                out.append(s)
            else:
                words = s.split()
                # greedily pack words by character budget approx
                cur, cur_len = [], 0
                char_budget = max_tokens * 4  # inverse of our heuristic
                for w in words:
                    if cur and cur_len + len(w) + 1 > char_budget:
                        out.append(" ".join(cur))
                        cur, cur_len = [w], len(w)
                    else:
                        cur.append(w); cur_len += len(w) + 1
                if cur:
                    out.append(" ".join(cur))
    return with_overlap(out, overlap_tokens=overlap_tokens)


In [2]:
!pip install spacy


In [35]:
from spacy.lang.en import English
nlp = English()
nlp.add_pipe("sentencizer")
doc = nlp("This is a sentence. This is another sentence.")
assert len(list(doc.sents)) == 2
from tqdm.auto import tqdm
import pandas as pd

from tqdm.auto import tqdm
import pandas as pd

def split_into_sentences(pages, nlp):
    """Return [(page_num, [sent1, sent2,...])]"""
    results = []
    for p in tqdm(pages, desc="Splitting into sentences"):
        doc = nlp(p["text"])
        sents = [s.text.strip() for s in doc.sents if s.text.strip()]
        results.append((p["Page_number"], sents))
    return results

def chunk_by_sentences(page_sents, chunk_size=10):
    """Group sentences into fixed-size chunks."""
    chunks = []
    for page_num, sents in page_sents:
        for i in range(0, len(sents), chunk_size):
            group = sents[i:i+chunk_size]
            text = " ".join(group)
            chunks.append({
                "page_number": page_num,
                "chunk_index": (i // chunk_size) + 1,
                "chunk_sentence_count": len(group),
                "chunk_word_count": sum(len(s.split()) for s in group),
                "chunk_char_count": len(text),
                "chunk_text": text
            })
    return chunks

# ---- run it ----
page_sents = split_into_sentences(pages_and_texts, nlp)
chunks_10sent = chunk_by_sentences(page_sents, chunk_size=10)

df_chunks = pd.DataFrame(chunks_10sent)
print(df_chunks.shape)
df_chunks.head()













Splitting into sentences:   0%|          | 0/1167 [00:00<?, ?it/s]

(1803, 6)


,page_number,chunk_index,chunk_sentence_count,chunk_word_count,chunk_char_count,chunk_text
0,1,1,4,117,745,Special Thanks to Bill Chismar – University of...
1,2,1,3,16,88,PART I CHAPTER 1. BASIC CONCEPTS IN NUTRITION ...
2,4,1,1,43,246,Image by Jim Hollyer / CC BY 4.0 Introduction ...
3,5,1,10,176,1155,Learning Objectives By the end of this chapter...
4,6,1,10,149,987,Macronutrients Nutrients that are needed in la...


In [36]:
random.sample(pages_and_texts, k=1)

[{'Page_number': 870,
  'Page_char_count': 977,
  'page_word_count': 147,
  'page_sentence_count_raw': 8,
  'page_token_count': 244,
  'text': 'often mimic their behavior and eating habits. Parents must continue to help their school-aged children and adolescents establish healthy eating habits and attitudes toward food. Their primary role is to bring a wide variety of health-promoting foods into the home, so that their children can make good choices. Learning Activities Technology Note: The second edition of the Human Nutrition Open Educational Resource (OER) textbook features interactive learning activities. These activities are available in the web-based textbook and not available in the downloadable versions (EPUB, Digital PDF, Print_PDF, or Open Document). Learning activities may be used across various mobile devices, however, for the best user experience it is strongly recommended that users complete these activities using a desktop or laptop computer and in Google Chrome. An inte

In [37]:
import random
df_chunks = pd.DataFrame(chunks_10sent)
print(df_chunks.shape)
df_chunks.head()

(1803, 6)


,page_number,chunk_index,chunk_sentence_count,chunk_word_count,chunk_char_count,chunk_text
0,1,1,4,117,745,Special Thanks to Bill Chismar – University of...
1,2,1,3,16,88,PART I CHAPTER 1. BASIC CONCEPTS IN NUTRITION ...
2,4,1,1,43,246,Image by Jim Hollyer / CC BY 4.0 Introduction ...
3,5,1,10,176,1155,Learning Objectives By the end of this chapter...
4,6,1,10,149,987,Macronutrients Nutrients that are needed in la...
